In [1]:
import sys
import os
import time
sys.path.append('..')

In [2]:
from awsdet.utils.misc.config import Config
from awsdet.utils.logger import get_root_logger
from awsdet.utils.runner.dist_utils import init_dist
from awsdet.datasets.data_generator import DataGenerator
from awsdet.datasets import build_dataset
from awsdet.datasets.loader.build_loader import build_dataloader
import tensorflow as tf
import matplotlib.pyplot as plt

from awsdet.datasets.data_generator import DataGenerator

In [3]:
#config_path = '/workspace/shared_workspace/deep-learning-models/configs/rpn/rpn_r50_fpn_1x_coco.py'
config_path = '/workspace/shared_workspace/deep-learning-models/configs/mask_rcnn/mask_rcnn_r50_fpn_1x_coco.py'
cfg = Config.fromfile(config_path)
timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
cfg.work_dir = os.path.join('./work_dirs',
                                os.path.splitext(os.path.basename(config_path))[0])
log_file = os.path.join(cfg.work_dir, f'{timestamp}.log')

In [4]:
init_dist()

In [5]:
coco = build_dataset(cfg.data.train)

loading annotations into memory...
Done (t=22.61s)
creating index...
index created!


In [6]:
#coco[0]['gt_masks'].to_tensor()

In [7]:
dg = DataGenerator(coco)()

In [8]:
img, meta, gt_boxes, gt_labels, gt_masks = next(dg)

Starting new loop for GPU: 0


In [9]:
train_tdf, train_size = build_dataloader(coco, 2)
train_tdf_iter = iter(train_tdf)

In [10]:
imgs, img_metas, gt_bboxes, gt_labels, gt_masks = next(train_tdf_iter)

Starting new loop for GPU: 0


In [11]:
import matplotlib.patches as patches

def get_original_image(img):
    img = (img * \
       [58.395, 57.12, 57.375] + \
       [123.675, 116.28, 103.53])/255.
    #img = img[:img_instance['img_metas']['img_shape'][0], :img_instance['img_metas']['img_shape'][1], :]
    return img

def show_box(img, bbox):
    img = get_original_image(img)
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(img)
    y1 = bbox[1]
    x1 = bbox[0]
    h = bbox[3] - bbox[1]
    w = bbox[2] - bbox[0]
    rect = patches.Rectangle((y1,x1),h,w,linewidth=1,edgecolor='r',facecolor='none')
    ax.add_patch(rect)
    plt.show()

In [102]:
#fig, ax = plt.subplots(figsize=(10, 10))
#ax.imshow(get_original_image(imgs[1]))

In [103]:
#fig, ax = plt.subplots(figsize=(10, 10))
#ax.imshow(gt_masks[1][0])

In [14]:
gt_labels

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 6,  9,  9],
       [ 3,  0, -1]], dtype=int32)>

In [15]:
#logger = get_root_logger(log_file=log_file, log_level=cfg.log_level)

In [16]:
coco.pipeline.transforms[-1].keys

['img', 'gt_bboxes', 'gt_labels', 'gt_masks']

In [17]:
#show_box(coco[0])

In [18]:
from awsdet.models.builder import build_backbone, build_neck

In [19]:
from awsdet.core.anchor.builder import build_anchor_generator
from awsdet.core.bbox.builder import build_assigner

In [20]:
#from awsdet.core.bbox.iou_calculators.iou2d_calculator import compute_overlaps
#from awsdet.utils.misc.meta import trim_zeros, calc_img_shapes
#from awsdet.core.bbox.assigners.assign_result

In [21]:
backbone = build_backbone(cfg.model.backbone)
neck = build_neck(cfg.model.neck)

In [34]:
imgs, img_metas, gt_bboxes, gt_labels, gt_masks = next(train_tdf_iter)
C2, C3, C4, C5 = backbone(imgs)
P2, P3, P4, P5, P6 = neck((C2, C3, C4, C5))

In [35]:
anchor_generator = build_anchor_generator(cfg.model.rpn_head.anchor_generator)

In [36]:
rpn_feature_maps = [P2, P3, P4, P5, P6]

In [37]:
featmap_sizes = [tf.shape(i)[1:3] for i in rpn_feature_maps]
num_imgs = len(img_metas)
multi_level_anchors = anchor_generator.grid_anchors(featmap_sizes)
anchor_list = [multi_level_anchors for _ in range(num_imgs)]
valid_flags = [anchor_generator.valid_flags(featmap_sizes, i[3:5]) for i in img_metas]

In [38]:
assigner_dict = {'type': 'MaxIoUAssigner',
                 'pos_iou_thr': 0.7,
                 'neg_iou_thr': 0.3}
assigner = build_assigner(assigner_dict)

In [65]:
assign_result = assigner.assign(anchor_list[0], gt_bboxes[0], valid_flags=valid_flags[0], gt_labels=gt_labels[0])

In [66]:
pos_anchors = tf.squeeze(tf.gather(tf.concat(anchor_list[0], axis=0), tf.where(assign_result.gt_inds>0)))
neg_anchors = tf.squeeze(tf.gather(tf.concat(anchor_list[0], axis=0), tf.where(assign_result.gt_inds==0)))

In [67]:
#tf.gather(result.labels, tf.where(result.gt_inds>0))

In [75]:
#show_box(imgs[0], pos_anchors[16])
import tensorflow as tf
from awsdet.utils.misc.meta import trim_zeros

In [88]:
anchors = tf.concat(anchor_list[0], axis=0)
gt_box, gt_bool = trim_zeros(gt_bboxes[0])
gt_flags = tf.zeros(tf.shape(anchors)[0], dtype=tf.int8)
bboxes = tf.concat([gt_box, anchors], axis=0)
gt_label = tf.boolean_mask(gt_labels[0], gt_bool)

In [99]:
self_inds = tf.range(
            1, len(gt_label) + 1, dtype=tf.int32)

In [100]:
self_inds

<tf.Tensor: shape=(7,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7], dtype=int32)>

In [98]:
gt_labels

<tf.Tensor: shape=(2, 24), dtype=int32, numpy=
array([[32,  2,  2,  0,  0, 35, 34, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1],
       [56, 56,  0,  0,  0,  0, 44, 44, 55, 55, 55, 55, 55, 55, 55, 42,
        42, 60, 44, 60, 42, 44, 42, 42]], dtype=int32)>